In [4]:
import pandas as pd
df = pd.read_csv('top_rated_wines.csv')
df=df[df['variety'].notna()]
data=df.sample(700).to_dict('records')
len(data)

700

In [5]:
data

[{'name': 'Georges Vernay Cote-Rotie Maison Rouge 2016',
  'region': 'Cote Rotie, Rhone, France',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'The very rugged vineyard makes any mechanization impossible. The vines are maintained manually. Phytosanitary treatments are made with organic products'},
 {'name': 'Chateau La Mission Haut-Brion 2015',
  'region': 'Pessac-Leognan, Bordeaux, France',
  'variety': 'Red Wine',
  'rating': 96.0,
  'notes': 'The color is quite deep with garnet-red highlights. The nose proves complex from the very first. The oak shows through, but not excessively so, and is well-integrated. The bouquet features hints of chocolate and liquorice when the wine is swirled in the glass, showing how ripe the grapes were.  2015 La Mission starts out with fresh fruit flavors and a dense tannic structure that caresses the palate. It picks up power as it develops towards a long aftertaste with mocha and chocolate nuances. 2015 is unquestionably one of the greatest vi

In [8]:
from qdrant_client import QdrantClient,models
from sentence_transformers import SentenceTransformer

In [9]:
encoder=SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
qdrant=QdrantClient(":memory:")

In [12]:
qdrant.recreate_collection(collection_name="top_wines",
                           vectors_config=models.VectorParams(
                               size=encoder.get_sentence_embedding_dimension(),
                               distance=models.Distance.COSINE
                           )
                           )

/var/folders/xt/q0yq_jd90r963tpy0vpd8ggm0000gn/T/ipykernel_60699/1720626052.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(collection_name="top_wines",


True

In [13]:
qdrant.upload_points(collection_name='top_wines',
                     points=[
                         models.PointStruct(
                             id=idx,
                             vector=encoder.encode(doc["notes"]).tolist(),
                             payload=doc
                         ) for idx,doc in enumerate(data)
                     ]
                     )

In [15]:
hits=qdrant.search(collection_name="top_wines",
                   query_vector=encoder.encode("A wine from Mendoza Argentina").tolist(),
                   limit=3
                   )
for hit in hits:
    print(hit.payload,"score:",hit.score)

{'name': 'Jorge Ordonez Number 4 Esencia (375ML half-bottle) 2004', 'region': 'Spain', 'variety': 'Boutique', 'rating': 99.0, 'notes': 'Esencia is a unique wine that incorporates the raisined muscat grape.  After 24 months in barrel, we achieve a partial fermentation of the must.  Alois Kracher, through this wine, sought to convey the essence of the village of Almáchar, in the heart of the Axarquía, famous from time immemorial for its delicious muscat grapes and raisins.  '} score: 0.5802860229602369
{'name': 'Kistler Vineyards Kistler Vineyard Pinot Noir 1998', 'region': 'Sonoma Valley, Sonoma County, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'A concentrated wine, with impressive energy to its red and black fruit flavors and subtle earth tones.'} score: 0.5738881789016652
{'name': 'Domaine Bois de Boursan Chateauneuf-du-Pape Cuvee Felix 2000', 'region': 'Chateauneuf-du-Pape, Rhone, France', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'It is a structured and powe